In [1]:
########################start##############################
import pandas as pd
df=pd.read_csv('./datas/Data_MergeNew.csv',index_col=0)
df.index=pd.to_datetime(df.date,format='%Y-%m-%d') 
df.drop('date',axis=1,inplace=True)
df_train=df.loc['2020-01-07':'2021-04-02'] 
df_test=df.loc['2021-04-02':'2021-04-23']

name=['cityin','cityto','incity','city']
name1=['cityin','cityto','incity']
x_train=df_train.drop(name,axis=1).values  
y_train=df_train[name1].values
x_test=df_test.drop(name,axis=1).values  
y_test=df_test[name1].values
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

#####################标准化##############################
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(x_train)
x_train = ss.transform(x_train) 
x_test = ss.transform(x_test)

(57504, 43) (19133, 43) (57504, 3) (19133, 3)


In [2]:
import csv
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei',font_scale=1.5)  # 解决Seaborn中文显示问题并调整字体大小
sns.set_style("white")
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:                   
        os.makedirs(path)            
        print("---  new folder...  ---")
        print("---  OK  ---")
    else:
        print("---  There is this folder!  ---")
mkdir("./results_merge/")        
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error,r2_score,mean_squared_log_error

def calculate(y_true, y_predict, n, p):
    y_true = y_true
    y_predict = y_predict
    mse = mean_squared_error(y_true, y_predict)
    rmse = np.sqrt(mean_squared_error(y_true, y_predict))
    mae = mean_absolute_error(y_true, y_predict)
    r2 = r2_score(y_true, y_predict)
    mad = median_absolute_error(y_true, y_predict)
    mape = np.mean(np.abs((y_true - y_predict) / y_true)) * 100
    r2_adjusted = 1-((1-r2)*(n-1))/(n-p-1)
    # print('MSE: ', mse)
    # print('RMSE: ', rmse)
    # print('MAE: ', mae)
    # print('R2: ', r2)
    # print('MAD:', mad)
    # print('MAPE:', mape)
    # print('R2_Adjusted: ',r2_adjusted)
    return mse,rmse,mae,r2,mad,mape,r2_adjusted
# 将参数和评估结果写入文件
def write_csv_result(path_1,path_2,all_metrics,all_parameter):
    with open(path_1,"a",encoding="utf-8",newline="")as f:
        f = csv.writer(f)
        f.writerow(all_metrics)
    with open(path_2,"a",encoding="utf-8",newline="")as f:
        f = csv.writer(f)
        f.writerow(all_parameter)


---  There is this folder!  ---


In [3]:
from sklearn.neighbors import KNeighborsRegressor
import csv
import os
import numpy as np
from sklearn.multioutput import RegressorChain

def Knn(train_x,test_x,train_y,test_y,name):
    path_a = "./results_merge/"  + "Knn_" + name + "_" + "assess.csv"
    path_p = "./results_merge/"  + "Knn_" + name + "_" + "parameter.csv"
    path_b = "./results_merge/"  + "Knn_" + name + "_" + "best.csv"
    model_path = "./results_merge/"  + "Knn_" + name + "_" + "best.m"
    # 人工设置网格搜索的范围
    n_neighbors = [3,5,7,9] # 默认为5
    weights = ['uniform', 'distance']
    algorithm = ["brute","kd_tree","ball_tree"]
    leaf_size = [25,30,35] #默认是30
    metric = ["euclidean","manhattan","chebyshev","minkowski","wminkowski","seuclidean","mahalanobis"]
    P = [1,2] # 只在 wminkowski 和 minkowski 调
    all_nb = len(n_neighbors) * len(weights) * len(algorithm) * len(leaf_size) * len(metric) * len(P)
    num=1
    # 用于重启训练模型，提高效率，不重复跑相同的实验
    if(os.path.exists(path_a)):
        data = pd.read_csv(path_a)
        if(data.shape[0]>1):
            nums = int(data.values[-1,0])
        else:
            nums = 0
    else:
        nums = 0 
        col_a = ['num','mse','rmse','mae','r2','mad','mape','r2_adjusted','rmsle']
        col_p = ['num','metric','algorithm','weights','n_neighbors','leaf_size','P']
        write_csv_result(path_a,path_p,col_a,col_p)
    # 用于保存最好的模型
    if(os.path.exists(path_b)):
        best_result = 10*10**30
    else:
        with open(path_b,"a",encoding="utf-8",newline="")as f:
            f = csv.writer(f)
            f.writerow(['num','mse','rmse','mae','r2','mad','mape','r2_adjusted','rmsle'])
        best_result = 10*10**30
    # 网格搜索
    for n in n_neighbors:
        for l in leaf_size:
            for p in P:
                for a in algorithm:
                    for m in metric:
                        for w in weights:
                            if(nums>=num):
                                num = num+1
                            else:
                                try:
                                    if(m=="wminkowski" or m=="minkowski"):
                                        print("KNN start....{}/{}".format(num,all_nb))
                                        model = KNeighborsRegressor(n_neighbors=n,leaf_size=l,p=p,weights=w,metric=m,algorithm=a)
                                        
                                    else:
                                        print("KNN start....{}/{}".format(num,all_nb))
                                        model = KNeighborsRegressor(n_neighbors=n,leaf_size=l,weights=w,metric=m,algorithm=a)
                                    wrapper = RegressorChain(model)
                                    wrapper.fit(train_x,train_y)
                                    pred_test = wrapper.predict(test_x)
            #     
                                    y_test=test_y       
                                    y_test1=[]
                                    for i in y_test:
                                        y_test1.append(i[0])
                                    pred_test1=[]
                                    for i in pred_test:
                                        pred_test1.append(i[0])
                                    y_test2=[]
                                    for i in y_test:
                                        y_test2.append(i[1])
                                    pred_test2=[]
                                    for i in pred_test:
                                        pred_test2.append(i[1])
                                    y_test3=[]
                                    for i in y_test:
                                        y_test3.append(i[2])
                                    pred_test3=[]
                                    for i in pred_test:
                                        pred_test3.append(i[2])
                                    pred_test1 = np.array(pred_test1).reshape(-1,1)
                                    y_test1 = np.array(y_test1).reshape(-1,1)
                                    sample_n = pred_test1.shape[0]
                                    feature_n = x_test.shape[1]
                                    mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1 = calculate(y_test1,pred_test1,sample_n,feature_n)
                                

                                    pred_test2 = np.array(pred_test2).reshape(-1,1)
                                    y_test2 = np.array(y_test2).reshape(-1,1)
                                    sample_n = pred_test2.shape[0]
                                    feature_n = x_test.shape[1]
                                    mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2 = calculate(y_test2,pred_test2,sample_n,feature_n)
                                    

                                    pred_test3 = np.array(pred_test3).reshape(-1,1)
                                    y_test3 = np.array(y_test3).reshape(-1,1)
                                    sample_n = pred_test3.shape[0]
                                    feature_n = x_test.shape[1]
                                    mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3 = calculate(y_test3,pred_test3,sample_n,feature_n)
                    
                                    mse,rmse,mae,r2,mad,mape,r2_adjusted = (mse1+mse2+mse3)/3,(rmse1+rmse2+rmse3)/3,(mae1+mae2+mae3)/3,(r21+r22+r23)/3,(mad1+mad2+mad3)/3,(mape1+mape2+mape3)/3,(r2_adjusted1+r2_adjusted2+r2_adjusted3)/3
                                    all_m = [num,mse,rmse,mae,r2,mad,mape,r2_adjusted]
                                    all_p = [num,m,a,w,n,l,p]
                                    # print(all_m)
                                    write_csv_result(path_a,path_p,all_m,all_p)              
                                    if(rmse < best_result):
                                        joblib.dump(filename=model_path,value=model,compress=True)
                                        with open(path_b,"w",encoding="utf-8",newline="")as f:
                                            f = csv.writer(f)
                                            f.writerow([num,mse,rmse,mae,r2,mad,mape,r2_adjusted])
                                            f.writerow([num,mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1])
                                            f.writerow([num,mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2])
                                            f.writerow([num,mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3])
                                        best_result = rmse
                                        print("current best result, mse:{},mae:{},rmse:{},mad:{},r2:{},mape:{},r2 adjusted:{}".format(mse,mad,rmse,mad,r2,mape,r2_adjusted))
                                    print("end....",num)
                                    num = num+1
                                    print("--------------------------------") 
                                except:
                                    num = num+1
                                    print("error")
name='Migration'
Knn(x_train,x_test,y_train,y_test,name)


KNN start....384/1008
current best result, mse:0.2870933488433414,mae:0.13381875452187306,rmse:0.5329443657703136,mad:0.13381875452187306,r2:0.7118881424841087,mape:23.13608041214891,r2 adjusted:0.7112391399238289
end.... 384
--------------------------------
KNN start....385/1008
current best result, mse:0.26877028867847996,mae:0.12100786666666707,rmse:0.5163441755753538,mad:0.12100786666666707,r2:0.7325187203990248,mape:17.84484071496698,r2 adjusted:0.731916190406734
end.... 385
--------------------------------
KNN start....386/1008
current best result, mse:0.26632000187547666,mae:0.11797850638267011,rmse:0.5140302983951854,mad:0.11797850638267011,r2:0.7352464861652779,mape:17.52119424626254,r2 adjusted:0.7346501007550996
end.... 386
--------------------------------
KNN start....387/1008
error
KNN start....388/1008
error
KNN start....389/1008
end.... 389
--------------------------------
KNN start....390/1008
end.... 390
--------------------------------
KNN start....391/1008
end.... 39

end.... 515
--------------------------------
KNN start....516/1008
current best result, mse:0.2563651320151428,mae:0.11619574944035192,rmse:0.504722713643542,mad:0.11619574944035192,r2:0.7474627771085253,mape:17.351570509726738,r2 adjusted:0.7468939101912256
end.... 516
--------------------------------
KNN start....517/1008
end.... 517
--------------------------------
KNN start....518/1008
end.... 518
--------------------------------
KNN start....519/1008
end.... 519
--------------------------------
KNN start....520/1008
end.... 520
--------------------------------
KNN start....521/1008
end.... 521
--------------------------------
KNN start....522/1008
end.... 522
--------------------------------
KNN start....523/1008
end.... 523
--------------------------------
KNN start....524/1008
end.... 524
--------------------------------
KNN start....525/1008
end.... 525
--------------------------------
KNN start....526/1008
end.... 526
--------------------------------
KNN start....527/1008
erro

end.... 663
--------------------------------
KNN start....664/1008
end.... 664
--------------------------------
KNN start....665/1008
end.... 665
--------------------------------
KNN start....666/1008
end.... 666
--------------------------------
KNN start....667/1008
error
KNN start....668/1008
error
KNN start....669/1008
error
KNN start....670/1008
error
KNN start....671/1008
error
KNN start....672/1008
error
KNN start....673/1008
end.... 673
--------------------------------
KNN start....674/1008
end.... 674
--------------------------------
KNN start....675/1008
end.... 675
--------------------------------
KNN start....676/1008
end.... 676
--------------------------------
KNN start....677/1008
end.... 677
--------------------------------
KNN start....678/1008
end.... 678
--------------------------------
KNN start....679/1008
end.... 679
--------------------------------
KNN start....680/1008
end.... 680
--------------------------------
KNN start....681/1008
error
KNN start....682/1008


end.... 805
--------------------------------
KNN start....806/1008
end.... 806
--------------------------------
KNN start....807/1008
error
KNN start....808/1008
error
KNN start....809/1008
end.... 809
--------------------------------
KNN start....810/1008
end.... 810
--------------------------------
KNN start....811/1008
end.... 811
--------------------------------
KNN start....812/1008
end.... 812
--------------------------------
KNN start....813/1008
end.... 813
--------------------------------
KNN start....814/1008
end.... 814
--------------------------------
KNN start....815/1008
end.... 815
--------------------------------
KNN start....816/1008
end.... 816
--------------------------------
KNN start....817/1008
end.... 817
--------------------------------
KNN start....818/1008
end.... 818
--------------------------------
KNN start....819/1008
end.... 819
--------------------------------
KNN start....820/1008
end.... 820
--------------------------------
KNN start....821/1008
error


end.... 957
--------------------------------
KNN start....958/1008
end.... 958
--------------------------------
KNN start....959/1008
end.... 959
--------------------------------
KNN start....960/1008
end.... 960
--------------------------------
KNN start....961/1008
error
KNN start....962/1008
error
KNN start....963/1008
error
KNN start....964/1008
error
KNN start....965/1008
error
KNN start....966/1008
error
KNN start....967/1008
end.... 967
--------------------------------
KNN start....968/1008
end.... 968
--------------------------------
KNN start....969/1008
end.... 969
--------------------------------
KNN start....970/1008
end.... 970
--------------------------------
KNN start....971/1008
end.... 971
--------------------------------
KNN start....972/1008
end.... 972
--------------------------------
KNN start....973/1008
end.... 973
--------------------------------
KNN start....974/1008
end.... 974
--------------------------------
KNN start....975/1008
error
KNN start....976/1008
